In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.integrate import quad
from scipy import stats
from numpy.random import Generator, PCG64

import tqdm
import json
import inspect
import warnings
from IPython.display import display, Markdown, Latex

warnings.filterwarnings('ignore')

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

##### Config

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
def get_sample_from_distribution(n_sample, random_state, params):
    """
    Return a sample of size N from de distribution
    """
    scale, loc, b, a = params["scale"], params["loc"], params["b"], params["a"]
    #logger.info(f"param a: {a} \n param b: {b} \n param loc: {loc} \n param scale: {scale}")
    distribution = stats.johnsonsu(a=a, b=b, loc=loc, scale=scale)
    #distribution.random_state = random_state # this is hideen bcs needs set one time random state
    return distribution.rvs(size=n_sample) if not pd.isnull(n_sample) else 0

In [5]:
def get_expectation_value(params, n_sample):
    """
    Calcula la esperanza de la distribución Johnson SU.
    """
    scale, loc, b, a = params["scale"], params["loc"], params["b"], params["a"]
    distribution = stats.johnsonsu(a=a, b=b, loc=loc, scale=scale)

    # Utilizamos la función quad de SciPy para calcular la integral numérica
    expectation, _ = quad(lambda x: x * distribution.pdf(x), -np.inf, np.inf)
    logger.info(f"[Expected Value] {expectation*n_sample}")
    return expectation*n_sample

The expected value (mean) of a Johnson SU distribution is given by the formula:
$$
E[X] = \text{loc} + \frac{\text{scale}}{b} \left(1 - \frac{1}{\sqrt{a^2 + b^2}}\right) \sinh\left(\frac{1}{2} \ln\left(\frac{a}{b} + \sqrt{\frac{a^2}{b^2} + 1}\right)\right)
$$

# Data
## Daily Demand

In [6]:
raw = pd.read_csv("../../data/output_layer_process.csv")
logger.info(f'Quantity of rows input {len(raw.index)}')

INFO:__main__:Quantity of rows input 71774


In [7]:
display(Markdown(f"#### Quantity of customer by layer-period"))
pd.pivot_table(data=raw, index=["year","month"], columns=["layer"], values="cod_customer", aggfunc="count")

#### Quantity of customer by layer-period

layer            0     1      2
year month                     
2020 1      1421.0   9.0   42.0
     2      1253.0  14.0   46.0
     3      1057.0   4.0   29.0
     4       962.0   6.0   18.0
     5      1138.0   6.0   31.0
     6      1284.0   9.0   48.0
     7      1293.0  14.0   49.0
     8      1189.0   5.0   38.0
     9      1525.0   5.0   56.0
     10     1787.0   7.0   73.0
     11     1685.0  17.0   75.0
     12     2030.0  17.0   93.0
2021 1      1391.0   7.0   82.0
     2       385.0   NaN    9.0
     3      1734.0   3.0   68.0
     4      1683.0   5.0   69.0
     5      1530.0  13.0   64.0
     6      1473.0  11.0   46.0
     7      1732.0  12.0   75.0
     8      1795.0  13.0   71.0
     9      2055.0  13.0   87.0
     10     2339.0  20.0   91.0
     11     1896.0  15.0   97.0
     12     2513.0  26.0  112.0
2022 1      1760.0  17.0   83.0
     2      2495.0  18.0   99.0
     3      2594.0  13.0  107.0
     4      2459.0  13.0   86.0
     5      2214.0  20.0   86.0
     6      2831.0  15.0  127.0
     7      2558.0  24.0   93.0
     8      2534.0  22.0   92.0
     9      2758.0  22.0  128.0
     10     2998.0  26.0  134.0
     11     2853.0  29.0  122.0
     12     3293.0  37.0  144.0

## Params Distribution

In [8]:
f = open('../../data/scenario-generation/distribution_params.json')
params = json.load(f)
#params

## Customer per Pixel-Month related to 2022

In [9]:
display(Markdown(f"Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses"))
pivot_tbl = pd.pivot_table(data=raw[(raw.year==2022)], index=["pixel"], columns=["layer"], values="cod_customer", aggfunc="nunique")
display(pivot_tbl.T)

Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses

pixel,26.0,42.0,71.0,72.0,73.0,87.0,88.0,104.0,105.0,106.0,107.0,108.0,109.0,111.0,115.0,116.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,168.0,169.0,170.0,171.0,172.0,173.0,177.0,178.0,179.0,180.0,181.0,182.0,183.0,184.0,185.0,188.0,189.0,192.0,193.0,194.0,195.0,196.0,197.0,198.0,199.0,200.0,201.0,208.0,209.0,210.0,211.0,212.0,213.0,214.0,215.0,216.0,217.0,218.0,224.0,226.0,227.0,228.0,229.0,230.0,234.0,235.0,241.0,242.0,244.0,245.0,246.0,258.0,259.0,261.0,262.0,263.0,274.0,275.0,279.0,280.0,290.0,291.0
layer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,3.0,12.0,25.0,36.0,10.0,7.0,6.0,1.0,8.0,1.0,11.0,1.0,2.0,11.0,3.0,6.0,59.0,77.0,74.0,43.0,49.0,54.0,22.0,30.0,34.0,56.0,36.0,2.0,1.0,35.0,54.0,21.0,60.0,1.0,5.0,62.0,63.0,11.0,39.0,44.0,12.0,45.0,40.0,47.0,40.0,40.0,21.0,3.0,12.0,8.0,9.0,2.0,26.0,165.0,96.0,40.0,45.0,18.0,20.0,50.0,8.0,7.0,27.0,1.0,25.0,128.0,226.0,171.0,84.0,98.0,81.0,42.0,28.0,4.0,32.0,10.0,96.0,184.0,335.0,132.0,120.0,104.0,65.0,35.0,18.0,57.0,78.0,88.0,109.0,85.0,96.0,66.0,40.0,35.0,6.0,4.0,14.0,48.0,60.0,46.0,117.0,20.0,3.0,1.0,3.0,58.0,4.0,68.0,86.0,30.0,6.0,11.0,25.0,14.0,34.0,3.0,1.0,11.0,12.0,3.0
1,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,1.0,NaN,NaN,3.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,3.0,3.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0,2.0,NaN,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,2.0,1.0,1.0,3.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,7.0,5.0,4.0,4.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,3.0,5.0,NaN,NaN,6.0,4.0,NaN,6.0,1.0,1.0,2.0,1.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,4.0,5.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,7.0,15.0,7.0,4.0,3.0,3.0,1.0,NaN,1.0,2.0,8.0,22.0,41.0,19.0,5.0,7.0,6.0,1.0,NaN,3.0,4.0,4.0,7.0,1.0,4.0,3.0,NaN,1.0,1.0,1.0,NaN,3.0,2.0,1.0,7.0,NaN,NaN,NaN,NaN,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,1.0,NaN,NaN


## A) Layer Low

In [10]:
PERIODS = [i for i in range(1,13)]
LAYER="low"

In [11]:
df_low = pd.DataFrame(pivot_tbl[0]).reset_index()
df_low.rename(columns={
        0: "n_customers",
    }, inplace=True
)
df_low["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_low[period] = df_low.apply(lambda x: get_expectation_value(
                                            params = params[LAYER][str(period)],
                                            n_sample = int(x["n_customers"])
                                            ),
                                            axis=1
                                 )

INFO:__main__:[PERIOD] 1
INFO:__main__:[Expected Value] 8.839952483718147
INFO:__main__:[Expected Value] 8.839952483718147
INFO:__main__:[Expected Value] 35.35980993487259
INFO:__main__:[Expected Value] 73.66627069765121
INFO:__main__:[Expected Value] 106.07942980461775
INFO:__main__:[Expected Value] 29.466508279060484
INFO:__main__:[Expected Value] 20.62655579534234
INFO:__main__:[Expected Value] 17.679904967436293
INFO:__main__:[Expected Value] 2.9466508279060486
INFO:__main__:[Expected Value] 23.57320662324839
INFO:__main__:[Expected Value] 2.9466508279060486
INFO:__main__:[Expected Value] 32.41315910696653
INFO:__main__:[Expected Value] 2.9466508279060486
INFO:__main__:[Expected Value] 5.893301655812097
INFO:__main__:[Expected Value] 32.41315910696653
INFO:__main__:[Expected Value] 8.839952483718147
INFO:__main__:[Expected Value] 17.679904967436293
INFO:__main__:[Expected Value] 173.85239884645688
INFO:__main__:[Expected Value] 226.89211374876575
INFO:__main__:[Expected Value] 218.

In [12]:
df_low

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12
0,26.0,3.0,low,8.839952,8.955627,8.997029,9.020307,8.965099,9.559477,9.022743,9.017655,9.433766,9.576791,9.692920,10.475674
1,42.0,3.0,low,8.839952,8.955627,8.997029,9.020307,8.965099,9.559477,9.022743,9.017655,9.433766,9.576791,9.692920,10.475674
2,71.0,12.0,low,35.359810,35.822508,35.988117,36.081229,35.860398,38.237909,36.090972,36.070621,37.735066,38.307165,38.771681,41.902697
3,72.0,25.0,low,73.666271,74.630226,74.975245,75.169226,74.709162,79.662311,75.189526,75.147126,78.614721,79.806594,80.774336,87.297286
4,73.0,36.0,low,106.079430,107.467525,107.964352,108.243686,107.581194,114.713728,108.272917,108.211862,113.205198,114.921496,116.315044,125.708091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,275.0,3.0,low,8.839952,8.955627,8.997029,9.020307,8.965099,9.559477,9.022743,9.017655,9.433766,9.576791,9.692920,10.475674
117,279.0,1.0,low,2.946651,2.985209,2.999010,3.006769,2.988366,3.186492,3.007581,3.005885,3.144589,3.192264,3.230973,3.491891
118,280.0,11.0,low,32.413159,32.837299,32.989108,33.074460,32.872031,35.051417,33.083391,33.064736,34.590477,35.114901,35.540708,38.410806
119,290.0,12.0,low,35.359810,35.822508,35.988117,36.081229,35.860398,38.237909,36.090972,36.070621,37.735066,38.307165,38.771681,41.902697


## B) Layer Medium

In [13]:
PERIODS = [i for i in range(1,13)]
LAYER="medium"

In [14]:
df_medium = pd.DataFrame(pivot_tbl[2]).reset_index()
df_medium.rename(columns={
        2: "n_customers",
    }, inplace=True
)
df_medium = df_medium.dropna()
df_medium["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_medium[period] = df_medium.apply(lambda x: get_expectation_value(
                                            params = params[LAYER][str(period)],
                                            n_sample = int(x["n_customers"])
                                            ),
                                            axis=1
                                 )

INFO:__main__:[PERIOD] 1
INFO:__main__:[Expected Value] 60.92168081709528
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 91.38252122564292
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 30.46084040854764
INFO:__main__:[Expected Value] 213.22588285983346
INFO:__main__:[Expected Value] 152.3042020427382
INFO:__main__:[Expected Value] 121.84336163419056
INFO:__main__:[Expected Value] 121.84336163419056
INFO:__main__:[Expected Value] 121.84336163419056
INFO:__main__:[Expected Value] 60.92168081709528
INFO:__main__:[Expected Value] 60.92168081709528
INFO:__main__:[Expected Value] 91.38252122564292
INFO:__main__:[Expected Value] 152.30420

In [15]:
df_medium

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12
1,42.0,2.0,medium,60.921681,57.301186,59.705503,60.518215,57.669911,60.322163,57.820005,60.890498,58.041275,59.426431,55.549118,59.849862
2,71.0,1.0,medium,30.460840,28.650593,29.852751,30.259107,28.834956,30.161081,28.910002,30.445249,29.020638,29.713216,27.774559,29.924931
3,72.0,1.0,medium,30.460840,28.650593,29.852751,30.259107,28.834956,30.161081,28.910002,30.445249,29.020638,29.713216,27.774559,29.924931
4,73.0,3.0,medium,91.382521,85.951780,89.558254,90.777322,86.504867,90.483244,86.730007,91.335747,87.061913,89.139647,83.323677,89.774794
6,88.0,1.0,medium,30.460840,28.650593,29.852751,30.259107,28.834956,30.161081,28.910002,30.445249,29.020638,29.713216,27.774559,29.924931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,245.0,4.0,medium,121.843362,114.602373,119.411005,121.036429,115.339823,120.644326,115.640010,121.780996,116.082551,118.852862,111.098237,119.699725
109,246.0,5.0,medium,152.304202,143.252966,149.263756,151.295537,144.174778,150.805407,144.550012,152.226245,145.103188,148.566078,138.872796,149.624656
114,263.0,1.0,medium,30.460840,28.650593,29.852751,30.259107,28.834956,30.161081,28.910002,30.445249,29.020638,29.713216,27.774559,29.924931
115,274.0,4.0,medium,121.843362,114.602373,119.411005,121.036429,115.339823,120.644326,115.640010,121.780996,116.082551,118.852862,111.098237,119.699725


## C) Layer high

In [16]:
PERIODS = [i for i in range(1,13)]
LAYER="high"

In [17]:
df_high = pd.DataFrame(pivot_tbl[1]).reset_index()
df_high.rename(columns={
        1: "n_customers",
    }, inplace=True
)
df_high = df_high.dropna()
df_high["layer"] = LAYER
for period in PERIODS:
    logger.info(f'[PERIOD] {period}')
    # set RANDOM STATE
    random_state = Generator(PCG64(12345))
    # sampling by pixel
    df_high[period] = df_high.apply(lambda x: get_expectation_value(
                                            params = params[LAYER][str(period)],
                                            n_sample = int(x["n_customers"])
                                            ),
                                            axis=1
                                 )

INFO:__main__:[PERIOD] 1
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 281.7295233573795
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 281.7295233573795
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 93.9098411191265
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__:[Expected Value] 187.819682238253
INFO:__main__

In [18]:
df_high

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12
1,42.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
4,73.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
17,120.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292
18,121.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
19,122.0,3.0,high,281.729523,273.128510,257.168017,263.115337,261.917573,279.650121,283.048888,279.265039,269.426806,273.238927,300.246043,296.055877
24,127.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292
30,135.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
32,137.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292
33,138.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292
36,141.0,3.0,high,281.729523,273.128510,257.168017,263.115337,261.917573,279.650121,283.048888,279.265039,269.426806,273.238927,300.246043,296.055877


## Merge Data

In [19]:
df_output = pd.concat([
    df_low,
    df_medium,
    df_high,
    ]
).reset_index(drop=True)
df_output

,pixel,n_customers,layer,1,2,3,4,5,6,7,8,9,10,11,12
0,26.0,3.0,low,8.839952,8.955627,8.997029,9.020307,8.965099,9.559477,9.022743,9.017655,9.433766,9.576791,9.692920,10.475674
1,42.0,3.0,low,8.839952,8.955627,8.997029,9.020307,8.965099,9.559477,9.022743,9.017655,9.433766,9.576791,9.692920,10.475674
2,71.0,12.0,low,35.359810,35.822508,35.988117,36.081229,35.860398,38.237909,36.090972,36.070621,37.735066,38.307165,38.771681,41.902697
3,72.0,25.0,low,73.666271,74.630226,74.975245,75.169226,74.709162,79.662311,75.189526,75.147126,78.614721,79.806594,80.774336,87.297286
4,73.0,36.0,low,106.079430,107.467525,107.964352,108.243686,107.581194,114.713728,108.272917,108.211862,113.205198,114.921496,116.315044,125.708091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,199.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
226,211.0,2.0,high,187.819682,182.085674,171.445345,175.410225,174.611715,186.433414,188.699259,186.176693,179.617870,182.159285,200.164029,197.370585
227,213.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292
228,229.0,1.0,high,93.909841,91.042837,85.722672,87.705112,87.305858,93.216707,94.349629,93.088346,89.808935,91.079642,100.082014,98.685292


In [20]:
df_output.groupby("layer").count()

,pixel,n_customers,1,2,3,4,5,6,7,8,9,10,11,12
layer,,,,,,,,,,,,,,
high,34,34,34,34,34,34,34,34,34,34,34,34,34,34
low,121,121,121,121,121,121,121,121,121,121,121,121,121,121
medium,75,75,75,75,75,75,75,75,75,75,75,75,75,75


In [21]:
df_output.to_csv('../../data/scenario-generation/scenario-expected-value.csv', index=False)